In [ ]:
import os
import shutil
import chess
import random
from stockfish import Stockfish

# Stockfish initialization

In [ ]:
KAGGLE = True

if KAGGLE:
    input_stockfish_path = "/kaggle/input/stockfish_ubuntu/other/default/1/stockfish-ubuntu-x86-64-avx2"
    stockfish_path = "/kaggle/working/stockfish"
    shutil.copy(input_stockfish_path, stockfish_path)
    os.chmod(stockfish_path, 0o755)
else:
    stockfish_path = "../../models/validation_stockfish/stockfish-windows-x86-64-avx2.exe"
    
stockfish = Stockfish(path=stockfish_path)
stockfish.set_skill_level(20)


# How to analyze the moves

In [ ]:
board = chess.Board()

def analyze_move(board, stockfish):
    stockfish.set_fen_position(board.fen())
    board_score_before = stockfish.get_evaluation()['value']

    move = random.choice(list(board.legal_moves))
    board.push(move)
    
    stockfish.set_fen_position(board.fen())
    board_score_after = stockfish.get_evaluation()['value']

    # division by 100 converts from pawn to centipawn score, while subtracting 0.01 is penalization for each move, to win as fast as possible
    reward = board_score_after / 100 - board_score_before / 100 - 0.01

    return reward, board_score_after, board_score_before, move

print(analyze_move(board, stockfish))

# How to analyze models with stockfish

In [ ]:
def configure_stockfish_using_skill_level(skill_level):
    stockfish = Stockfish(path=stockfish_path)
    stockfish.set_skill_level(skill_level)
    return stockfish

def configure_stockfish_using_rating(elo_rating):
    stockfish = Stockfish(path=stockfish_path)
    stockfish.set_elo_rating(elo_rating)
    return stockfish

def play_single_game(agent, stockfish, max_moves=50):
    board = chess.Board()
    stockfish.set_fen_position(board.fen())

    for move_count in range(max_moves):
        if board.turn:
            move = agent.select_action(board)
        else:
            move = chess.Move.from_uci(stockfish.get_best_move())
        board.push(move)
        stockfish.set_fen_position(board.fen())

        if board.is_game_over():
            break

    return board.result()

def evaluate_agent(agent, skill_levels, games_per_rating=20):
    results = []

    for level in skill_levels:
        stockfish = configure_stockfish_using_skill_level(level)
        outcomes = {"Wins": 0, "Losses": 0, "Draws": 0}

        for _ in range(games_per_rating):
            result = play_single_game(agent, stockfish)
            if result == "1-0":
                outcomes["Wins"] += 1
            elif result == "0-1":
                outcomes["Losses"] += 1
            else:
                outcomes["Draws"] += 1

        win_rate = (outcomes["Wins"] + 0.5 * outcomes["Draws"]) / games_per_rating
        outcomes["Win Rate"] = win_rate
        outcomes["Stockfish Level"] = level
        results.append(outcomes)

        print(f"Elo {level}: {outcomes}")

    return results

agent = None
stockfish_skill_levels = [1, 2, 3, 4] 
results = evaluate_agent(agent, stockfish_skill_levels)